torch.manual_seed(1024) 是 PyTorch 中的一个函数调用，用于设定随机数生成器的种子为 1024。这样做的目的是为了确保在每次运行程序时（如权重初始化、数据打乱等操作中使用随机数的地方）生成的随机数序列是固定的，从而使实验结果具有可重复性。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as DataSet
import torch.utils.data as DataLoader
import math
from dataclasses import dataclass
import numpy as np
import torch_directml
device = torch_directml.device()
torch.manual_seed(1024)



block_size

假设我们在训练一个语言模型，模型一次输入一个文本序列。如果 block_size 设置为 512，则表示每个输入序列最多包含 512 个 token。比如说，一段文章有 1024 个 token，那么模型会把它拆成两个 512 token 的小块进行处理。

batch_size

如果 batch_size 为 12，则每次训练模型时，会同时处理 12 个序列。举个例子，如果你有 120 个样本数据，那么整个训练过程中就会经过 10 次批处理（每批 12 个样本）。

n_layer

n_layer 表示模型中 Transformer 块（层）的数量。假设设置为 12，就意味着数据会依次经过 12 层这样的转换，每一层都可以提取更深层次的特征。比如第一层可能只捕捉到基本的词语关系，而到了第 12 层，模型可能理解到更复杂的语义信息。

n_head

n_head 表示每个 Transformer 层中的多头注意力机制的头数。假设设为 12，意味着在一层中会有 12 个并行的注意力头，每个头可以学习不同角度的上下文关系。例如，对于一个单词“apple”，不同的注意力头可能会分别关注这个词前后的不同信息，从而共同决定它在句中的意义。

hidden_dim

hidden_dim: 这个参数通常用来指定 Transformer 模型中前馈神经网络 (Feed-Forward Network, FFN) 中间的隐藏层维度。在标准的 Transformer 结构中，每个 Transformer 块都包含一个多头注意力层和一个位置独立的前馈网络。这个前馈网络通常由两个线性层组成，第一个线性层将输入维度（通常是 n_embd）扩展到一个更大的hidden_dim，然后通过一个激活函数（如 ReLU 或 GeLU），第二个线性层再将其映射回原来的维度 (n_embd)。

n_embd

n_embd 指的是嵌入向量的维度。如果设置为 768，那么每个 token 都会被转换为一个 768 维的向量。比如单词“apple”可能在嵌入空间里表示为一个由 768 个数字组成的向量，这些数字代表了“apple”的语义特征。

dropout

dropout 是一种防止模型过拟合的技术。当设置为 0.1 时，意味着在网络训练过程中，每层中的一部分神经元（大约 10%）会随机被“关闭”，不参与计算。就好比在开会时随机选择部分人暂时不发言，这样可以防止某个固定人员的信息过度影响讨论，帮助模型更好地推广到未知数据上。

head_size

head_size 是每个注意力头处理的向量的维度，由公式 n_embd // n_head 计算得出，此处 768 // 12 = 64。举例来说，每个注意力头就只专注于 64 维的信息，相比整个 768 维向量，这样分割后每个头都能更专注地捕捉特定的细粒度信息。

vocab_size

vocab_size 表示模型所认识的所有 token 的总数。如果设置为 50257，则说明整个词汇表中有 50,257 个不同的词或者子词单位。比如，当模型处理一段文本时，它会将每个 token 转换为对应的索引，该索引必须在 0 到 50256 之间，以确保模型能正确处理所有可能出现的 token。

# 多头注意力机制 (Multi-Head Attention) 详解

多头注意力机制是 Transformer 模型中的一个核心组件，它允许模型同时从输入的不同表示子空间中学习信息。

## 例子：理解指代关系

考虑以下句子：

> **"The animal didn't cross the street because it was too tired."**

模型需要理解这里的 "it" 指的是 "The animal" 而不是 "the street"。多头注意力机制在这里就能发挥作用。

### 模型配置示例

假设我们的模型配置如下：

*   `n_embd = 768` (嵌入维度)
*   `n_head = 12` (注意力头数)
*   `head_size = 64` (每个头的维度, 即 `n_embd / n_head`)

### 不同注意力头的作用

当模型处理到单词 "it" 时，它会计算 "it" 与句子中其他所有单词的注意力分数。由于有 12 个注意力头 (`n_head = 12`)，这个计算会并行进行 12 次，每个头可能关注不同的方面：

*   **头 1 (指代消解头):** 可能专门解决指代问题。它会给 "animal" 一个高注意力分数，给 "street" 一个低分数。
*   **头 2 (句法结构头):** 可能关注句法关系，例如识别 "it" 是从句的主语。
*   **头 3 (相邻词关联头):** 可能只关注 "it" 附近的词，如 "because" 和 "was"。
*   **头 4 (动词-主语关系头):** 可能关注动词 "was tired" 和其主语 "it" 的关系。
*   **其他 8 个头:** 可能关注其他模式，如位置信息、词性搭配、否定词影响等。

### 计算过程

1.  **拆分/投影 (Split/Project):** 输入给 "it" 的 768 维向量（以及其他词的向量）会被分别进行 12 次线性变换，为每个头生成独立的 64 维的查询 (Query, Q)、键 (Key, K) 和值 (Value, V) 向量。
2.  **独立计算注意力 (Independent Attention Calculation):** 每个头使用自己的 Q, K, V 向量独立计算注意力权重。例如，头 1 会计算出 "it" 对 "animal" 的高权重。
3.  **加权求和 (Weighted Sum):** 每个头根据自己计算出的权重，对其 V 向量进行加权求和，得到该头自己的 64 维输出向量。
4.  **合并 (Concatenate):** 将所有 12 个头的 64 维输出向量拼接起来，形成一个 12 * 64 = 768 维的最终输出向量。这个向量融合了来自 12 个不同注意力角度的信息。

## 优势

通过这种多头机制，模型能够从多个不同的表示子空间中捕捉信息，综合不同角度的分析（如指代关系、句法结构、局部上下文等），从而对输入（例如单词 "it"）获得更全面、更准确的理解。


In [2]:
@dataclass
class GPTConfig:
    block_size: int = 512
    batch_size: int = 12
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    hidden_dim: int = n_embd
    dropout: float = 0.1
    head_size: int = n_embd // n_head
    vocab_size: int = 50257

# Query (Q), Key (K), 和 Value (V) 的关系

在注意力机制（尤其是 Transformer 模型中的自注意力）中，Query (Q), Key (K), 和 Value (V) 是三个核心概念，它们协同工作来计算一个序列中不同部分之间的相关性，并据此生成新的表示。

想象一个场景：你在图书馆（代表输入序列）里寻找关于某个特定主题（代表当前处理的词/token）的信息。

1.  **Query (Q):** 代表你当前的**查询**或**问题**。也就是你正在处理的那个词/token，它想要知道序列中哪些其他部分与它最相关。在图书馆的比喻中，这就是你的研究主题。
    *   *目的：* 发起信息请求。

2.  **Key (K):** 代表序列中每个词/token 的**“标签”**或**“索引关键词”**。它与 Query 进行比较，以确定相关性。在图书馆的比喻中，这就像每本书的书名或目录关键词，你可以用你的研究主题（Query）去匹配这些关键词（Keys）。
    *   *目的：* 被查询，用于匹配。

3.  **Value (V):** 代表序列中每个词/token 的**实际内容**或**信息**。一旦 Query 和某个 Key 匹配成功（即找到了相关性），对应的 Value 就会被提取出来。在图书馆的比喻中，这就是书的实际内容。当你通过书名（Key）找到相关的书后，你真正需要的是书里的内容（Value）。
    *   *目的：* 提供信息。

## 它们的关系和工作流程：

1.  **生成 Q, K, V:** 对于输入序列中的每一个词/token，模型都会通过独立的线性变换（就像你之前看到的 `nn.Linear` 层）从其原始表示（比如词嵌入）生成对应的 Q, K, V 三个向量。重要的是，**同一个输入词会生成它自己的 Q, K, V**。
2.  **计算注意力分数:** 当前词的 **Query (Q)** 会和**序列中所有词**的 **Key (K)** 进行计算（通常是点积运算），得出一个分数。这个分数衡量了当前词（由 Q 代表）与序列中其他每个词（由 K 代表）的**相关性**或**相似度**。
3.  **归一化分数:** 这些分数通常会经过缩放（防止梯度消失/爆炸）和 Softmax 函数处理，转换成一组**权重**，所有权重加起来等于 1。高分数的词获得高权重，表示更相关。
4.  **加权求和:** 使用这些归一化后的权重，对**序列中所有词**的 **Value (V)** 进行**加权求和**。这意味着，与当前词（Query）更相关的词（通过 Q-K 匹配得到高权重），其对应的 Value 信息将在最终的输出中占更大的比重。
5.  **输出:** 这个加权求和的结果，就是当前词/token 经过注意力机制计算后得到的新表示。这个新表示融合了序列中所有相关词的信息，侧重于那些与当前词最相关的部分。

## 总结:

Q 是提问者，K 是被匹配的标签，V 是要提取的内容。Q 和 K 互动计算出相关性权重，然后用这些权重去加权聚合所有 V，得到最终的输出。这种机制使得模型能够动态地关注输入序列的不同部分，捕捉长距离依赖关系。

# Q, K, V 矩阵计算示例

这个例子演示了如何使用矩阵运算来计算 Query (Q), Key (K), 和 Value (V)，以及后续的注意力分数和最终输出。

假设我们有一个包含两个词的输入序列："词1", "词2"。

**1. 输入嵌入 (Input Embeddings)**

假设每个词的嵌入向量维度 `d_model` 为 4。我们将输入序列表示为一个矩阵 `X`，其中每一行代表一个词的嵌入：

```
X = [[1, 0, 1, 0],  # 词1 的嵌入
     [0, 1, 0, 1]]  # 词2 的嵌入
```
`X` 的形状是 `(序列长度, d_model)` = `(2, 4)`。

**2. 权重矩阵 (Weight Matrices)**

我们需要三个权重矩阵 `W_Q`, `W_K`, `W_V` 来将输入嵌入投影到 Q, K, V 空间。假设我们希望 Q, K, V 的维度 `d_k` = `d_v` = 3。那么这些权重矩阵的形状将是 `(d_model, d_k)` 或 `(d_model, d_v)`，即 `(4, 3)`。

为了简单起见，我们定义如下权重矩阵：

```
W_Q = [[1, 0, 1],
       [1, 0, 0],
       [0, 1, 1],
       [0, 1, 0]]

W_K = [[0, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 0, 1]]

W_V = [[0, 2, 0],
       [1, 0, 1],
       [1, 1, 0],
       [0, 0, 1]]
```

**3. 计算 Q, K, V 矩阵**

现在，我们将输入矩阵 `X` 与每个权重矩阵相乘，得到 Q, K, V 矩阵：

*   **Q = X @ W_Q** (矩阵乘法)
    ```
    Q = [[1, 0, 1, 0],  @  [[1, 0, 1],
         [0, 1, 0, 1]]       [1, 0, 0],
                             [0, 1, 1],
                             [0, 1, 0]]

      = [[1, 1, 2],  # 词1 的 Query 向量 q1
         [1, 1, 0]]  # 词2 的 Query 向量 q2
    ```
    `Q` 的形状是 `(序列长度, d_k)` = `(2, 3)`。

*   **K = X @ W_K**
    ```
    K = [[1, 0, 1, 0],  @  [[0, 1, 1],
         [0, 1, 0, 1]]       [1, 1, 0],
                             [1, 0, 1],
                             [0, 0, 1]]

      = [[1, 1, 2],  # 词1 的 Key 向量 k1
         [1, 1, 1]]  # 词2 的 Key 向量 k2
    ```
    `K` 的形状是 `(序列长度, d_k)` = `(2, 3)`。

*   **V = X @ W_V**
    ```
    V = [[1, 0, 1, 0],  @  [[0, 2, 0],
         [0, 1, 0, 1]]       [1, 0, 1],
                             [1, 1, 0],
                             [0, 0, 1]]

      = [[1, 3, 0],  # 词1 的 Value 向量 v1
         [1, 0, 2]]  # 词2 的 Value 向量 v2
    ```
    `V` 的形状是 `(序列长度, d_v)` = `(2, 3)`。

**4. 计算注意力分数 (Attention Scores)**

分数通过计算 Q 和 K 的转置 (`K^T`) 的点积得到：`Scores = Q @ K^T`。

```
K^T = [[1, 1],
       [1, 1],
       [2, 1]]  # 形状为 (d_k, 序列长度) = (3, 2)

Scores = Q @ K^T
       = [[1, 1, 2],  @  [[1, 1],
          [1, 1, 0]]       [1, 1],
                          [2, 1]]

       = [[6, 4],  # 词1 对 词1/词2 的原始注意力分数
          [2, 2]]  # 词2 对 词1/词2 的原始注意力分数
```
`Scores` 矩阵的形状是 `(序列长度, 序列长度)` = `(2, 2)`。

**5. 缩放和 Softmax (Scaling and Softmax)**

为了稳定梯度，我们通常将分数除以 \(\sqrt{d_k}\)。这里 `d_k = 3`，\(\sqrt{3} \approx 1.732\)。

```
Scaled_Scores = Scores / 1.732
              ≈ [[3.46, 2.31],
                 [1.16, 1.16]]
```

然后应用 Softmax 函数到每一行，得到注意力权重矩阵 `A`：

```python
# 示例计算第一行 Softmax:
# exp(3.46) ≈ 31.8
# exp(2.31) ≈ 10.1
# softmax(row1) = [31.8 / (31.8 + 10.1), 10.1 / (31.8 + 10.1)] ≈ [0.76, 0.24]

# 示例计算第二行 Softmax:
# exp(1.16) ≈ 3.19
# softmax(row2) = [3.19 / (3.19 + 3.19), 3.19 / (3.19 + 3.19)] ≈ [0.5, 0.5]

A = softmax(Scaled_Scores)
  ≈ [[0.76, 0.24],  # 词1 的注意力权重 (关注词1: 76%, 关注词2: 24%)
     [0.50, 0.50]]  # 词2 的注意力权重 (关注词1: 50%, 关注词2: 50%)
```
`A` 的形状仍然是 `(序列长度, 序列长度)` = `(2, 2)`。

**6. 计算最终输出 (Final Output)**

最后，将注意力权重矩阵 `A` 与 Value 矩阵 `V` 相乘，得到最终的输出 `Z`：

```
Z = A @ V
  ≈ [[0.76, 0.24],  @  [[1, 3, 0],
     [0.50, 0.50]]       [1, 0, 2]]

  ≈ [[(0.76*1 + 0.24*1), (0.76*3 + 0.24*0), (0.76*0 + 0.24*2)],
     [(0.50*1 + 0.50*1), (0.50*3 + 0.50*0), (0.50*0 + 0.50*2)]]

  ≈ [[1.00, 2.28, 0.48],  # 词1 经过注意力计算后的新表示 z1
     [1.00, 1.50, 1.00]]  # 词2 经过注意力计算后的新表示 z2
```
输出 `Z` 的形状是 `(序列长度, d_v)` = `(2, 3)`。每一行是对应输入词经过自注意力计算后得到的新向量表示，它融合了整个序列的信息，并根据计算出的注意力权重进行了侧重。

# `forward` 方法计算过程详解示例

我们用一个简单的例子来逐步解释这个 `forward` 方法的计算过程。

**假设配置：**
*   `head_size = 2` (每个注意力头的维度)
*   `block_size = 5` (最大序列长度，用于掩码)
*   `dropout = 0.0` (为了简化，假设训练时 dropout 概率为 0 或处于评估模式)

**假设输入 `x`：**
一个批次 (batch_size=1)，包含 3 个词 (seq_len=3)，每个词的嵌入维度是 4 (d_model=4)。

```python
# x 的形状为 (1, 3, 4)
# 例子中的数值是示意性的
# x = [[[...], [...], [...], [...]],  # 词 1 的 4 维嵌入
#      [[...], [...], [...], [...]],  # 词 2 的 4 维嵌入
#      [[...], [...], [...], [...]]]  # 词 3 的 4 维嵌入
```

---

## 1. 获取尺寸

```python
batch_size, seq_len, d_model = x.size()
# batch_size = 1
# seq_len = 3
# d_model = 4
```

---

## 2. 计算 Q, K, V

输入 `x` 分别通过 `self.key`, `self.query`, `self.value` 三个线性层（输入维度 4，输出维度 `head_size=2`）。

```python
k = self.key(x)     # 形状变为 (1, 3, 2)
q = self.query(x)    # 形状变为 (1, 3, 2)
v = self.value(x)    # 形状变为 (1, 3, 2)

# 假设经过线性层计算后得到（数值是编的，用于演示）：
# q = [[[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]]]  # 形状 (1, 3, 2)
# k = [[[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]]]  # 形状 (1, 3, 2)
# v = [[[1.3, 1.4], [1.5, 1.6], [1.7, 1.8]]]  # 形状 (1, 3, 2)
```

---

## 3. 计算原始注意力分数 `weight = q @ k.transpose(-2, -1)`

计算 Query 和 Key 的点积。需要将 Key 的最后两个维度转置。

*   `k.transpose(-2, -1)` 的形状是 `(1, 2, 3)`。
*   `q @ k.transpose(-2, -1)` 进行矩阵乘法：`(1, 3, 2) @ (1, 2, 3)` 结果形状为 `(1, 3, 3)`。

```python
# k.transpose(-2, -1) = [[[0.7, 0.9, 1.1],
#                         [0.8, 1.0, 1.2]]]

# weight = q @ k.transpose(-2, -1)
# weight ≈ [[[0.23, 0.29, 0.35],   # q1 对 k1, k2, k3 的分数
#            [0.53, 0.67, 0.81],   # q2 对 k1, k2, k3 的分数
#            [0.83, 1.05, 1.27]]]  # q3 对 k1, k2, k3 的分数
# 形状 (1, 3, 3)
```
这个 `weight` 矩阵的 `weight[0, i, j]` 表示第 `i` 个词的 Query 和第 `j` 个词的 Key 之间的原始相关性分数。

---

## 4. 应用注意力掩码 `weight.masked_fill(...)`

这里通常应用**因果掩码 (causal mask)**，防止当前位置关注到未来位置。我们假设需要一个下三角掩码（实际实现可能不同，但效果类似）。

```python
# 假设 self.attention_mask[:seq_len, :seq_len] 是一个因果掩码
# mask = [[1, 0, 0],
#         [1, 1, 0],
#         [1, 1, 1]]  # 形状 (3, 3)

# weight.masked_fill(mask == 0, float("-inf"))
# 将 mask 中为 0 的位置（上三角，不包括对角线）在 weight 中填充为负无穷

# weight ≈ [[[0.23, -inf, -inf],
#            [0.53, 0.67, -inf],
#            [0.83, 1.05, 1.27]]]
```
填充 `-inf` 的目的是，在接下来的 Softmax 计算中，这些位置的概率会趋近于 0。

---

## 5. Softmax 和缩放 `weight = F.softmax(weight, dim=-1) / math.sqrt(self.head_size)`

对最后一维（`dim=-1`）应用 Softmax，将分数转换为概率分布。**注意：** 代码在此处是在 Softmax **之后** 进行缩放 `1 / math.sqrt(head_size)`，这与原始 Transformer 论文不同。我们遵循代码的逻辑。`head_size = 2`, \(\sqrt{2} \approx 1.414\)。

```python
# 对每一行应用 Softmax:
# row1: softmax([0.23, -inf, -inf]) ≈ [1.0, 0.0, 0.0]
# row2: softmax([0.53, 0.67, -inf]) ≈ [0.465, 0.535, 0.0]
# row3: softmax([0.83, 1.05, 1.27]) ≈ [0.236, 0.292, 0.472]




# softmax_weight ≈ [[[1.0,   0.0,    0.0],
#                   [0.465, 0.535,  0.0],
#                   [0.236, 0.292,  0.472]]]

# 再除以 sqrt(head_size)
# weight = softmax_weight / 1.414
# weight ≈ [[[0.707, 0.0,   0.0],
#            [0.329, 0.378, 0.0],
#            [0.167, 0.206, 0.334]]]
```
现在 `weight` 包含了（经过缩放的）注意力权重。例如，`weight[0, 1, 0] ≈ 0.329` 表示在计算第 2 个词的输出时，来自第 1 个词的 Value 的贡献权重是 0.329。

---

## 6. Dropout `weight = self.dropout(weight)`

如果模型在训练模式且 `dropout > 0`，这一步会随机将 `weight` 中的一些元素设为 0，并对其他元素进行缩放。在评估模式或 `dropout = 0` 时，这一步不起作用。我们假设它不起作用。

---

## 7. 计算最终输出 `out = weight @ v`

将计算出的注意力权重 `weight` 与 Value 矩阵 `v` 相乘。

*   `weight` 形状 `(1, 3, 3)`
*   `v` 形状 `(1, 3, 2)`
*   `out = weight @ v` 结果形状为 `(1, 3, 2)`。

```python
# v = [[[1.3, 1.4], [1.5, 1.6], [1.7, 1.8]]]

# out = weight @ v

# out[0, 0, :] = 0.707 * v[0, 0, :] + 0.0 * v[0, 1, :] + 0.0 * v[0, 2, :]
#             ≈ [0.919, 0.990]

# out[0, 1, :] = 0.329 * v[0, 0, :] + 0.378 * v[0, 1, :] + 0.0 * v[0, 2, :]
#             ≈ [0.995, 1.066]

# out[0, 2, :] = 0.167 * v[0, 0, :] + 0.206 * v[0, 1, :] + 0.334 * v[0, 2, :]
#             ≈ [1.094, 1.165]

# out ≈ [[[0.919, 0.990],
#         [0.995, 1.066],
#         [1.094, 1.165]]]  # 形状 (1, 3, 2)
```
每一行 `out[0, i, :]` 是第 `i` 个输入词经过这个单头注意力计算后得到的新表示（维度为 `head_size=2`），它融合了来自序列中（根据注意力权重侧重的）其他词的 Value 信息。

---

## 8. 返回 `out`

函数返回计算得到的输出张量 `out`。


In [3]:
class SingleHeadAttention(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.head_size = config.head_size
        self.key = nn.Linear(config.hidden_dim, config.head_size)
        self.query = nn.Linear(config.hidden_dim, config.head_size)
        self.value = nn.Linear(config.hidden_dim, config.head_size)
        self.register_buffer(
            "attention_mask",
            torch.ones(config.block_size, config.block_size)
        )
        #这行代码创建并注册了一个名为 attention_mask 的方阵，
        # 大小与模型能处理的最大序列长度相匹配，初始值全为 1。 
        # 这个全 1 的矩阵通常不是最终使用的注意力掩码，而是作为基础
        self.dropout = nn.Dropout(config.dropout)
        # 创建一个 Dropout 层，用于在训练过程中随机丢弃一些神经元，
        # 以防止过拟合。dropout 是一个介于 0 和 1 之间的概率值，
        # 表示每个神经元被丢弃的概率。
        # 例如，如果 dropout 设置为 0.1，那么每个神经元有 10% 的概率被丢弃。
        # 在训练过程中，Dropout 层会随机选择一些神经元并将其权重设置为 0，
        # 从而迫使模型在训练过程中学习到更鲁棒的特征。
    def forward(self, x):
        device = torch_directml.device()
        batch_size, seq_len, d_model = x.size()
        # 获取输入张量 x 的批次大小、序列长度和模型维度
        # 例如，如果 x 的形状是 (batch_size, seq_len, d_model)，
        # 那么 batch_size 是输入样本的数量，seq_len 是序列长度，
        # d_model 是每个词的嵌入维度。
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        # 将输入张量 x 分别通过三个线性层 key, query, value，
        # 得到三个新的张量 k, q, v。
        # 这些新张量 k, q, v 的形状与 x 相同，
        # 其中 k 和 q 的形状是 (batch_size, seq_len, head_size)，
        # v 的形状是 (batch_size, seq_len, head_size)。
        #tril = torch.tril(torch.ones(seq_len, seq_len))
        weight = q @ k.transpose(-2, -1)
        weight = weight.masked_fill(self.attention_mask[:seq_len, :seq_len] == 0, float("-inf"))
        weight = F.softmax(weight, dim=-1) / math.sqrt(self.head_size)
        weight = self.dropout(weight)
        out = weight @ v
        return out

## MultiHeadAttention `__init__` 方法详解

这个 `__init__` 方法是 `MultiHeadAttention` 类的构造函数，负责初始化多头注意力机制所需的各个组件。

```python
class MultiHeadAttention(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                SingleHeadAttention(config)
                for _ in range(config.n_head)
            ]
        )
        self.proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
```

**逐行解释:**

1.  `def __init__(self, config: GPTConfig):`
    *   定义类的构造函数。
    *   接收一个 `GPTConfig` 类型的 `config` 对象作为参数，包含模型超参数。

2.  `super().__init__()`
    *   调用父类 `nn.Module` 的初始化方法，这是 PyTorch 自定义模块的标准做法。

3.  `self.heads = nn.ModuleList(...)`
    *   创建一个 `nn.ModuleList`，用于存储多个 `SingleHeadAttention` 实例。
    *   `[SingleHeadAttention(config) for _ in range(config.n_head)]`：列表推导式，根据 `config.n_head` (注意力头的数量) 创建相应数量的 `SingleHeadAttention` 实例。
    *   `self.heads` 最终成为一个包含多个独立注意力头模块的列表，PyTorch 可以正确管理这些模块。

4.  `self.proj = nn.Linear(config.hidden_dim, config.hidden_dim)`
    *   创建一个线性层 (`nn.Linear`)。
    *   它的作用是在 `forward` 方法中，将所有注意力头的输出拼接起来之后，进行一次最终的线性变换，将维度投影回模型的 `hidden_dim`。

5.  `self.dropout = nn.Dropout(config.dropout)`
    *   创建一个 `nn.Dropout` 层，丢弃概率由 `config.dropout` 指定。
    *   通常应用在多头注意力的最终输出上（经过 `self.proj` 之后），用于正则化。

### 具体例子

假设 `config` 如下：
*   `n_head = 12`
*   `n_embd = 768` (即 `hidden_dim`)
*   `head_size = 64`
*   `dropout = 0.1`

那么 `__init__` 方法执行后：

*   `self.heads` 会是一个 `ModuleList`，包含 **12 个** `SingleHeadAttention` 实例。每个实例内部都有自己的 Q, K, V 线性层（768 -> 64）。
*   `self.proj` 会是一个 **768 -> 768** 的 `nn.Linear` 层。
*   `self.dropout` 会是一个 `nn.Dropout` 层，其丢弃概率 `p` 为 **0.1**。

**总结:** 这个初始化过程设置了多头注意力的结构：创建了多个并行的单头注意力单元，并准备好了用于整合它们输出的投影层和正则化层。

## MultiHeadAttention `forward` 方法详解（含简化数值示例）

这个 `forward` 方法定义了多头注意力层如何处理输入数据 `x`。

```python
    def forward(self, x):
        # 1. 并行计算所有单头注意力的输出
        output = torch.cat(
            [h(x) for h in self.heads], # 对每个头调用 forward(x)
            dim=-1                     # 沿着最后一个维度拼接
        )
        # 2. 对拼接后的结果进行线性投影和 Dropout
        output = self.dropout(self.proj(output))
        # 3. 返回最终结果
        return output
```

### 方法解释:

1.  **并行计算单头输出**:
    *   输入 `x` 被并行地送入 `self.heads` 中的每一个 `SingleHeadAttention` 实例 (`h`)。
    *   每个头 `h` 计算并返回其独立的注意力输出。
2.  **拼接**:
    *   使用 `torch.cat(..., dim=-1)` 将所有单头注意力的输出张量沿着最后一个维度（特征维度）拼接起来。
3.  **线性投影和 Dropout**:
    *   将拼接后的结果通过 `self.proj` 线性层进行最终的变换。
    *   应用 `self.dropout` 层进行正则化（在训练时）。
4.  **返回结果**:
    *   返回处理后的最终输出张量。

### 具体例子与张量（数组）形状/值变化:

**简化配置:**

*   `batch_size = 1`
*   `seq_len = 2` (输入序列包含 2 个元素/词)
*   `n_embd = 4` (每个元素的嵌入维度)
*   `n_head = 2` (2 个注意力头)
*   `head_size = 2` (每个头的输出维度)
*   `dropout = 0.0` (禁用 Dropout 以简化)

**1. 输入 `x`**

假设输入 `x` (形状 `(1, 2, 4)`) 如下：

```python
# 假设这是两个词的嵌入向量
x = torch.tensor([[[1., 2., 3., 4.],  # 第一个词
                   [5., 6., 7., 8.]]]) # 第二个词
# 形状: (1, 2, 4)
```

**2. 并行计算单头输出 `[h(x) for h in self.heads]`**

我们有 2 个头 (`h_0`, `h_1`)。假设它们的输出 (形状 `(1, 2, 2)`) 如下：

*   `head_0_out = h_0(x)` (假设值):
    ```python
    head_0_out = torch.tensor([[[0.1, 0.2],  # 头 0, 词 1
                                [0.3, 0.4]]]) # 头 0, 词 2
    # 形状: (1, 2, 2)
    ```
*   `head_1_out = h_1(x)` (假设值):
    ```python
    head_1_out = torch.tensor([[[0.5, 0.6],  # 头 1, 词 1
                                [0.7, 0.8]]]) # 头 1, 词 2
    # 形状: (1, 2, 2)
    ```

**3. 拼接 `torch.cat(..., dim=-1)`**

将 `head_0_out` 和 `head_1_out` 沿最后一个维度拼接：

```python
output = torch.cat([head_0_out, head_1_out], dim=-1)

# 拼接结果:
# 词 1: [0.1, 0.2] + [0.5, 0.6] -> [0.1, 0.2, 0.5, 0.6]
# 词 2: [0.3, 0.4] + [0.7, 0.8] -> [0.3, 0.4, 0.7, 0.8]

output = torch.tensor([[[0.1, 0.2, 0.5, 0.6],
                        [0.3, 0.4, 0.7, 0.8]]])
# 形状: (1, 2, 4)
```

**4. 线性投影 `self.proj(output)` Dropout `self.dropout(proj_output)`** 

`self.proj` 是一个 `nn.Linear(4, 4)` 层。假设它对 `output` 进行变换后的结果如下：

## self.proj 和 self.dropout 详解（含矩阵示例）

我们接着上一个例子的结果，即多头注意力输出拼接后的张量 `output`：

```python
# 拼接后的输出 (来自 torch.cat)
output = torch.tensor([[[0.1, 0.2, 0.5, 0.6],  # 词 1 (拼接后)
                        [0.3, 0.4, 0.7, 0.8]]]) # 词 2 (拼接后)
# output 的形状: (1, 2, 4) - (batch_size, seq_len, hidden_dim)
```

### 1. `self.proj` (线性映射 / Linear Projection)

`self.proj` 是一个 `nn.Linear(4, 4)` 层。它内部包含一个权重矩阵 `W_proj` (4x4) 和一个偏置向量 `b_proj` (长度 4)。

**假设的权重和偏置：**

*   `W_proj` (权重矩阵, 形状 `(4, 4)`)：
    ```
    [[2., 0., 0., 0.],
     [0., 2., 0., 0.],
     [0., 0., 2., 0.],
     [0., 0., 0., 2.]]
    ```
    *(这是一个乘以 2 的单位矩阵，用于简化计算)*
*   `b_proj` (偏置向量, 形状 `(4,)`)：
    ```
    [0.1, 0.1, 0.1, 0.1]
    ```

**计算过程：**

线性层的计算是 `proj_output = output @ W_proj.T + b_proj`。

1.  **计算 `output @ W_proj.T`:** (由于 `W_proj` 对称, `W_proj.T = W_proj`)
    对 `output` 中的每个 4 维向量乘以 `W_proj`:
    *   词 1: `[0.1, 0.2, 0.5, 0.6] @ W_proj` = `[0.2, 0.4, 1.0, 1.2]`
    *   词 2: `[0.3, 0.4, 0.7, 0.8] @ W_proj` = `[0.6, 0.8, 1.4, 1.6]`
    结果矩阵 (形状 `(1, 2, 4)`):
    ```
    [[[0.2, 0.4, 1.0, 1.2],
      [0.6, 0.8, 1.4, 1.6]]]
    ```

2.  **加上偏置 `b_proj`:**
    将 `b_proj = [0.1, 0.1, 0.1, 0.1]` 加到上面结果的每个 4 维向量上:
    *   词 1: `[0.2, 0.4, 1.0, 1.2] + [0.1, 0.1, 0.1, 0.1]` = `[0.3, 0.5, 1.1, 1.3]`
    *   词 2: `[0.6, 0.8, 1.4, 1.6] + [0.1, 0.1, 0.1, 0.1]` = `[0.7, 0.9, 1.5, 1.7]`

**`proj_output` 结果 (形状 `(1, 2, 4)`)**:

```python
proj_output = torch.tensor([[[0.3, 0.5, 1.1, 1.3], # 词 1 经过线性层
                             [0.7, 0.9, 1.5, 1.7]]]) # 词 2 经过线性层
```

---

### 2. `self.dropout` (Dropout)

假设 Dropout 概率 `p = 0.5` (50% 概率丢弃)。Dropout 只在 **训练** 时生效。

*   **缩放因子：** `1 / (1 - p) = 1 / (1 - 0.5) = 2`。
*   **生成随机掩码 (假设)：** 为 `proj_output` 中的每个元素生成一个 0/1 掩码 (形状 `(1, 2, 4)`):
    ```
    mask = [[[1., 0., 0., 1.],  # 掩码 for 词 1
             [0., 1., 1., 0.]]] # 掩码 for 词 2
    ```
    *(0 表示丢弃, 1 表示保留)*

*   **应用掩码 (逐元素相乘)：** `masked_output = proj_output * mask`
    ```
    # masked_output =
    # [[[0.3*1, 0.5*0, 1.1*0, 1.3*1],
    #   [0.7*0, 0.9*1, 1.5*1, 1.7*0]]]
    # =
    masked_output = torch.tensor([[[0.3, 0.0, 0.0, 1.3],
                                   [0.0, 0.9, 1.5, 0.0]]])
    ```

*   **缩放保留的元素：** `final_output_train = masked_output * 缩放因子`
    ```python
    final_output_train = masked_output * 2
    # final_output_train =
    # [[[0.3*2, 0.0*2, 0.0*2, 1.3*2],
    #   [0.0*2, 0.9*2, 1.5*2, 0.0*2]]]
    # =
    final_output_train = torch.tensor([[[0.6, 0.0, 0.0, 2.6],
                                        [0.0, 1.8, 3.0, 0.0]]])
    ```

**最终输出:**

*   **训练时 (形状 `(1, 2, 4)`)**:
    ```python
    final_output_train = torch.tensor([[[0.6, 0.0, 0.0, 2.6],
                                        [0.0, 1.8, 3.0, 0.0]]])
    ```
*   **评估/推理时 (形状 `(1, 2, 4)`)**: Dropout 不生效，输出即为 `proj_output`。
    ```python
    final_output_eval = torch.tensor([[[0.3, 0.5, 1.1, 1.3],
                                       [0.7, 0.9, 1.5, 1.7]]])
    ```

这个例子演示了线性层如何通过矩阵运算变换数据，以及 Dropout 如何在训练时随机清零并缩放数据以实现正则化。

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                SingleHeadAttention(config)
                for _ in range(config.n_head)
            ]
        )
        self.proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x):
        output = torch.cat(
            [h(x) for h in self.heads],
            dim=-1
        )
        output = self.dropout(self.proj(output))
        return output

    
    
    

## FeedForward 网络详解（复杂数值示例）

下面是一个更接近实际情况的例子，演示 `FeedForward` 网络如何处理包含多个 Token 的序列，并应用了非零的 Dropout。

**设定:**

*   `batch_size = 1`
*   `seq_len = 2` (序列包含两个 Token)
*   `hidden_dim = 4` (输入/输出维度)
*   Intermediate dimension = `4 * hidden_dim = 16`
*   `dropout = 0.2` (Dropout 概率，假设处于 **训练模式**)

**1. 输入 `x`**

假设输入 `x` 是一个形状为 `(1, 2, 4)` 的张量：

```python
x = torch.tensor([[[ 1.0,  2.0, -1.0,  0.5],  # Token 1 嵌入
                   [-2.0,  1.0,  0.0,  1.5]]]) # Token 2 嵌入
# 形状: (1, 2, 4)
```

**2. 第一层线性变换（扩展层） `nn.Linear(4, 16)`**

*   **假设的权重 `W1` (形状 `(16, 4)`) 和偏置 `b1` (形状 `(16,)`)**:
    ```python
    # 使用一些特定的值
    W1 = torch.tensor([
        [ 1,  0,  0,  0], [ 0,  1,  0,  0], [ 0,  0,  1,  0], [ 0,  0,  0,  1], # 复制
        [-1,  0,  0,  0], [ 0, -1,  0,  0], [ 0,  0, -1,  0], [ 0,  0,  0, -1], # 取反
        [ 1,  1,  0,  0], [ 0,  1,  1,  0], [ 0,  0,  1,  1], [ 1,  0,  0,  1], # 混合 1
        [ 2,  0, -1,  0], [ 0,  2,  0, -1], [-1,  0,  2,  0], [ 0, -1,  0,  2]  # 混合 2
    ], dtype=torch.float32)
    b1 = torch.ones(16) * 0.1 # 偏置向量 [0.1, 0.1, ..., 0.1]
    ```
*   **计算 `output1 = x @ W1.T + b1`**:
    *   Token 1 (`[1.0, 2.0, -1.0, 0.5]`) -> `[1.1, 2.1, -0.9, 0.6, -0.9, -1.9, 1.1, -0.4, 3.1, 1.1, -0.4, 1.6, 3.1, 3.6, -1.9, -1.4]`
    *   Token 2 (`[-2.0, 1.0, 0.0, 1.5]`) -> `[-1.9, 1.1, 0.1, 1.6, 2.1, -0.9, 0.1, -1.4, -0.9, 1.1, 1.6, -0.4, -3.9, 0.6, 2.1, 2.1]`
*   **`output1` 结果 (形状 `(1, 2, 16)`)**:
    ```python
    output1 = torch.tensor([[
        [ 1.1,  2.1, -0.9,  0.6, -0.9, -1.9,  1.1, -0.4,  3.1,  1.1, -0.4,  1.6,  3.1,  3.6, -1.9, -1.4], # Token 1
        [-1.9,  1.1,  0.1,  1.6,  2.1, -0.9,  0.1, -1.4, -0.9,  1.1,  1.6, -0.4, -3.9,  0.6,  2.1,  2.1]  # Token 2
    ]])
    ```

**3. GELU 激活函数 `nn.GELU()`**

逐元素应用 GELU。下面是**近似/示意值**。

*   **`activated_output` (假设的近似结果，形状 `(1, 2, 16)`)**:
    ```python
    activated_output = torch.tensor([[
        # Token 1 (GELU applied) - Approximated
        [ 1.1,  2.1, -0.1,  0.6, -0.1, -0.0,  1.1, -0.1,  3.1,  1.1, -0.1,  1.6,  3.1,  3.6, -0.0, -0.0],
        # Token 2 (GELU applied) - Approximated
        [-0.0,  1.1,  0.1,  1.6,  2.1, -0.1,  0.1, -0.0, -0.1,  1.1,  1.6, -0.1, -0.0,  0.6,  2.1,  2.1]
    ]])
    ```

**4. 第二层线性变换（压缩层） `nn.Linear(16, 4)`**

*   **假设的权重 `W2` (形状 `(4, 16)`) 和偏置 `b2` (形状 `(4,)`)**:
    ```python
    # W2 设计为对 16 维输入的不同部分做加权平均
    W2 = torch.zeros(4, 16, dtype=torch.float32)
    W2[0, 0:4] = 0.25   # 输出1：平均前4个
    W2[1, 4:8] = 0.25   # 输出2：平均接下来4个
    W2[2, 8:12] = 0.25  # 输出3：平均再接下来4个
    W2[3, 12:16] = 0.25 # 输出4：平均最后4个
    b2 = torch.tensor([-0.05, 0.0, 0.05, 0.0]) # 偏置向量
    ```
*   **计算 `output2 = activated_output @ W2.T + b2`**:
    *   Token 1 (对 `activated_output` 的第一行加权平均 + 偏置) -> `[0.875, 0.225, 1.475, 1.675]`
    *   Token 2 (对 `activated_output` 的第二行加权平均 + 偏置) -> `[0.650, 0.525, 0.675, 1.200]`
*   **`output2` 结果 (形状 `(1, 2, 4)`)**:
    ```python
    output2 = torch.tensor([[
        [0.875, 0.225, 1.475, 1.675], # Token 1
        [0.650, 0.525, 0.675, 1.200]  # Token 2
    ]])
    ```

**5. Dropout `nn.Dropout(0.2)` (训练模式)**

*   **缩放因子:** `1 / (1 - 0.2) = 1.25`
*   **随机掩码 (假设):** 假设丢弃 Token 1 的第 3 个元素和 Token 2 的第 4 个元素。
    ```python
    mask = torch.tensor([[[1., 1., 0., 1.],  # Mask for Token 1
                          [1., 1., 1., 0.]]]) # Mask for Token 2
    ```
*   **应用掩码:** `masked_output = output2 * mask`
    ```python
    masked_output = torch.tensor([[
        [0.875, 0.225, 0.000, 1.675],
        [0.650, 0.525, 0.675, 0.000]
    ]])
    ```
*   **缩放:** `final_output_train = masked_output * 1.25`
    ```python
    final_output_train = torch.tensor([[
        [1.09375, 0.28125, 0.0,     2.09375], # Token 1 after dropout
        [0.8125,  0.65625, 0.84375, 0.0    ]  # Token 2 after dropout
    ]])
    ```

**最终输出 (训练时，形状 `(1, 2, 4)`)**

```python
final_output_train = torch.tensor([[[1.09375, 0.28125, 0.0,     2.09375],
                                   [0.8125,  0.65625, 0.84375, 0.0    ]]])
```
*(注意：在评估/推理模式下，Dropout 不生效，最终输出将是 `output2`)*

In [5]:
class FeedForward(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.hidden_dim,4 * config.hidden_dim),
            nn.GELU(),
            nn.Linear(4 * config.hidden_dim, config.hidden_dim),
            nn.Dropout(config.dropout)
        )    
    def forward(self, x):
        return self.net(x)

## Transformer Block `forward` 方法详解（复杂数组示例）

下面通过一个具体的数组（张量）例子，逐步演示数据如何流经一个 Transformer `Block` 的 `forward` 方法。

**设定:**

*   `batch_size = 1`
*   `seq_len = 2` (序列包含两个 Token)
*   `hidden_dim = 4` (输入/输出维度)
*   *(注意：`MultiHeadAttention` 和 `FeedForward` 的内部计算结果是假设的/示意性的，重点在于展示 `Block` 的数据流。)*

**1. 输入 `x`**

假设输入 `x` (形状 `(1, 2, 4)`) 是：

```python
x = torch.tensor([[[ 1.0,  2.0, -1.0,  0.5],  # Token 1
                   [-2.0,  1.0,  0.0,  1.5]]]) # Token 2
# 形状: (1, 2, 4)
```

**2. 第一个子层: `x = x + self.attn(self.ln1(x))`**

*   **a) `self.ln1(x)` (Layer Normalization 1):**
    对 `x` 应用层归一化。
    *   **假设的 `ln1_output` (形状 `(1, 2, 4)`)**:
        ```python
        # (示意值，表示归一化效果)
        ln1_output = torch.tensor([[[ 0.2,  1.2, -1.3, -0.1], # Token 1 (Normalized)
                                    [-1.5,  0.5, -0.5,  1.5]]]) # Token 2 (Normalized)
        ```

*   **b) `self.attn(ln1_output)` (Multi-Head Attention):**
    将归一化后的结果输入多头注意力层。
    *   **假设的 `attn_output` (形状 `(1, 2, 4)`)**:
        ```python
        # (示意值，表示注意力计算结果)
        attn_output = torch.tensor([[[ 0.5,  0.8, -0.2,  1.0], # Token 1 (Contextualized)
                                     [-1.0,  0.6,  0.3, -0.1]]]) # Token 2 (Contextualized)
        ```

*   **c) `x + attn_output` (第一个残差连接):**
    将注意力层的输出 `attn_output` 加回到子层的**原始输入** `x` 上。
    ```python
    #   [[[ 1.0,  2.0, -1.0,  0.5],        <- x (Token 1)
    #    [-2.0,  1.0,  0.0,  1.5]]]       <- x (Token 2)
    # + [[[ 0.5,  0.8, -0.2,  1.0],        <- attn_output (Token 1)
    #    [-1.0,  0.6,  0.3, -0.1]]]       <- attn_output (Token 2)
    # ===============================
    #   [[[ 1.5,  2.8, -1.2,  1.5],        <- residual1_output (Token 1)
    #    [-3.0,  1.6,  0.3,  1.4]]]       <- residual1_output (Token 2)

    residual1_output = torch.tensor([[[ 1.5,  2.8, -1.2,  1.5],
                                      [-3.0,  1.6,  0.3,  1.4]]])
    # 形状: (1, 2, 4)
    ```
    *(这个结果在代码中更新了变量 `x`)*

**3. 第二个子层: `x = x + self.ffn(self.ln2(x))`**

*   **a) `self.ln2(residual1_output)` (Layer Normalization 2):**
    对第一个子层的输出 `residual1_output` (现在存储在 `x` 中) 应用层归一化。
    *   **假设的 `ln2_output` (形状 `(1, 2, 4)`)**:
        ```python
        # (示意值，表示归一化效果)
        ln2_output = torch.tensor([[[ 0.1,  1.4, -1.1,  0.6], # Token 1 (Normalized)
                                    [-1.6,  0.7, -0.6,  0.5]]]) # Token 2 (Normalized)
        ```

*   **b) `self.ffn(ln2_output)` (Feed-Forward Network):**
    将归一化后的结果输入前馈网络层。
    *   **假设的 `ffn_output` (形状 `(1, 2, 4)`)**:
        ```python
        # (示意值，表示 FFN 处理结果)
        ffn_output = torch.tensor([[[ 0.7,  0.1, -0.5,  0.3], # Token 1 (Processed by FFN)
                                    [-0.4,  0.9,  0.1, -0.2]]]) # Token 2 (Processed by FFN)
        ```

*   **c) `residual1_output + ffn_output` (第二个残差连接):**
    将前馈网络的输出 `ffn_output` 加回到这个子层的**输入** `residual1_output` 上。
    ```python
    #   [[[ 1.5,  2.8, -1.2,  1.5],        <- residual1_output (Token 1)
    #    [-3.0,  1.6,  0.3,  1.4]]]       <- residual1_output (Token 2)
    # + [[[ 0.7,  0.1, -0.5,  0.3],        <- ffn_output (Token 1)
    #    [-0.4,  0.9,  0.1, -0.2]]]       <- ffn_output (Token 2)
    # ===============================
    #   [[[ 2.2,  2.9, -1.7,  1.8],        <- block_output (Token 1)
    #    [-3.4,  2.5,  0.4,  1.2]]]       <- block_output (Token 2)

    block_output = torch.tensor([[[ 2.2,  2.9, -1.7,  1.8],
                                  [-3.4,  2.5,  0.4,  1.2]]])
    # 形状: (1, 2, 4)
    ```
    *(这个结果再次更新了变量 `x`)*

**4. 返回 `block_output`**

`Block` 的 `forward` 方法最终返回 `block_output`。

```python
# 最终输出 (形状: (1, 2, 4))
final_output = torch.tensor([[[ 2.2,  2.9, -1.7,  1.8],
                              [-3.4,  2.5,  0.4,  1.2]]])
```

这个例子展示了数据在一个典型的 Transformer Block (Pre-LN 变体) 中的完整流动过程，包括层归一化、注意力、残差连接、前馈网络等步骤。

In [6]:
class Block(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.hidden_dim)
        self.attn = MultiHeadAttention(config)
        self.ln2 = nn.LayerNorm(config.hidden_dim)
        self.ffn = FeedForward(config)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

## GPT `forward` 方法详解（复杂数组示例）

下面通过一个具体的、包含数组（张量）的例子，详细演示您的 `GPT` 类 `forward` 方法的计算流程。

**重要提示:** 您在问题中提供的 `forward` 方法代码，在计算损失 (loss) 的 `if/else` 逻辑部分**存在错误**。在 `else` 分支中（对应 `targets is None` 的情况），代码仍然尝试执行 `targets = targets.view(...)`，这必然会导致 `AttributeError: 'NoneType' object has no attribute 'view'` 错误。

**我们将分两部分演示：**

1.  **按您当前代码的逻辑演示（当 `targets` 为 `None` 时）:** 展示到计算 `logits` 为止的流程，并指出 `else` 分支的问题。
2.  **按正确逻辑演示（当 `targets` 被提供时）:** 展示如果 `targets` 不为 `None`，损失计算应该如何进行（假设 `if/else` 逻辑已修正）。

---

### 场景 1: 按您当前代码逻辑 (`targets` is `None`)

**设定:**

*   `batch_size = 2`
*   `seq_len = 3`
*   `vocab_size = 10` (词汇表大小)
*   `n_embd = 4` (嵌入/隐藏维度)
*   `block_size = 5` (最大序列长度)
*   `n_layer = 1` (假设只有一个 Transformer Block)
*   *(子模块如 `Block`, `LayerNorm`, `Linear` 的内部计算结果是假设的/示意性的)*
*   **当前场景：调用时 `targets` 为 `None`**

**1. 输入 `idx`**

输入 `idx` 是一个形状为 `(2, 3)` 的整数张量。

```python
# 假设随机生成的输入 idx (形状: (2, 3))
idx = torch.tensor([[5, 2, 9],  # 第一个样本序列
                   [1, 0, 7]]) # 第二个样本序列
# 假设设备为 dml
```

**2. 获取维度 & 词嵌入**

*   `batch, seq_len = idx.size()` -> `batch = 2`, `seq_len = 3`
*   `token_embeddings = self.token_embedding(idx)`:
    `self.token_embedding` (假设是 `nn.Embedding(10, 4)`) 将索引映射到 4 维向量。
    *   **假设的 `token_embeddings` (形状 `(2, 3, 4)`)**:
        ```python
        token_embeddings = torch.tensor([
            [[ 0.1,  0.2, -0.1,  0.3], [ 0.4, -0.2,  0.5,  0.1], [ 0.9,  0.8, -0.7,  0.6]], # 样本 1
            [[-0.5,  0.6,  0.7, -0.8], [ 1.0,  1.1,  1.2,  1.3], [ 0.3, -0.4,  0.5, -0.6]]  # 样本 2
        ])
        ```

**3. 位置编码**

*   `pos = torch.arange(seq_len, device=idx.device)` -> `tensor([0, 1, 2])` (形状 `(3,)`)
*   `pos_embeddings = self.pos_emb(pos)`:
    `self.pos_emb` (假设是 `nn.Embedding(5, 4)`) 将位置索引映射到 4 维向量。
    *   **假设的 `pos_embeddings` (形状 `(3, 4)`)**:
        ```python
        pos_embeddings = torch.tensor([
            [ 0.01,  0.02,  0.03,  0.04], # 位置 0
            [ 0.05,  0.06,  0.07,  0.08], # 位置 1
            [ 0.09,  0.10,  0.11,  0.12]  # 位置 2
        ])
        ```

**4. 合并嵌入 `x = token_embeddings + pos_emb`**

词嵌入和位置编码相加（广播）。
*   **计算 `x` (形状 `(2, 3, 4)`)**:
    ```python
    x = torch.tensor([
        [[ 0.11,  0.22,  0.02,  0.34], [ 0.45, -0.14,  0.57,  0.18], [ 0.99,  0.90, -0.59,  0.72]], # 样本 1 + pos_emb
        [[-0.49,  0.62,  0.73, -0.76], [ 1.05,  1.16,  1.27,  1.38], [ 0.39, -0.30,  0.61, -0.48]]  # 样本 2 + pos_emb
    ])
    ```

**5. `targets = None` (根据您代码第 20 行)**

`targets` 变量被显式赋值为 `None`。

**6. 通过 Transformer 块 `x = self.blocks(x)`**

输入 `x` 经过 `Block` 处理。
*   **假设的 `blocks_output` (形状 `(2, 3, 4)`)**:
    ```python
    # 经过注意力、FFN 等处理后的结果
    blocks_output = torch.tensor([
        [[ 1.2,  0.8, -0.5,  1.1], [ 0.9,  1.5, -0.2,  0.7], [ 0.5, -0.1,  1.3,  0.9]], # 样本 1
        [[-1.1,  0.4,  1.8, -0.6], [-0.7,  1.2,  0.9,  1.4], [ 1.0,  0.2, -0.8,  0.3]]  # 样本 2
    ])
    x = blocks_output # 更新 x
    ```

**7. 最终层归一化 `x = self.ln_final(x)`**

对 `blocks_output` 应用最终的 Layer Normalization。
*   **假设的 `ln_final_output` (形状 `(2, 3, 4)`)**:
    ```python
    # 经过最终 LN 后的结果
    ln_final_output = torch.tensor([
        [[ 1.0,  0.5, -1.2,  0.7], [ 0.6,  1.3, -0.8,  0.1], [-0.2, -0.9,  1.5,  0.6]], # 样本 1
        [[-0.9, -0.1,  1.6, -0.6], [-0.5,  0.8,  0.4,  1.1], [ 0.8, -0.3, -1.1,  0.5]]  # 样本 2
    ])
    x = ln_final_output # 更新 x
    ```

**8. 计算 Logits `logits = self.lm_head(x)`**

`self.lm_head` (假设是 `nn.Linear(4, 10)`) 将 4 维向量映射到 10 维 logits。
*   **假设的 `logits` (形状 `(2, 3, 10)`)**:
    ```python
    # 假设经过线性变换后的 logits
    logits = torch.tensor([
        [[-0.1, 1.2, ..., 0.5], [-0.4, 0.8, ..., -0.1], [1.5, -0.9, ..., 0.2]], # 样本 1, 位置 0, 1, 2 logits
        [[ 0.3, -1.1, ..., 0.9], [ 0.6, 0.2, ..., 1.4], [-0.8, 1.7, ..., -0.3]]  # 样本 2, 位置 0, 1, 2 logits
    ])
    ```

**9. 损失计算 (`if/else` 分支 - 按您当前错误代码)**

*   `loss = None` (初始化)
*   `if targets is not None:` -> **条件为 False** (因为第 20 行 targets=None)
*   代码**错误地进入 `else` 分支**。
*   `batch, seq_len, vocab_size = logits.size()` -> (2, 3, 10)
*   `logits = logits.view(batch * seq_len, vocab_size)` -> `logits` 变形为 `(6, 10)`
*   `targets = targets.view(batch * seq_len)` -> **!!! 尝试在 None 对象上调用 .view() -> 导致 AttributeError !!!**

**返回值 (如果代码没崩溃):** `logits` (可能变形为 `(6,10)`) 和 `loss=None`。

---

### 场景 2: 按正确逻辑演示 (`targets` is provided)

现在假设我们**修正了 `forward` 方法的 `if/else` 逻辑**，并且在调用时提供了 `targets`。

**修正后的 `forward` 逻辑 (损失计算部分):**

```python
        # ... (计算 logits 的部分不变) ...
        logits = self.lm_head(x) # (B, T, vocab_size)

        loss = None
        if targets is not None: # <--- 正确的检查
            # 只有 targets 不为 None 时才执行
            B, T, C = logits.shape
            logits_for_loss = logits.view(B * T, C) # (B*T, C)
            targets_for_loss = targets.view(B * T) # (B*T)
            loss = nn.functional.cross_entropy(logits_for_loss, targets_for_loss) # 计算损失

        return logits, loss # 返回原始 logits 和计算出的 loss (或 None)
```

**假设调用时提供了 `targets` (形状 `(2, 3)`)**:

```python
targets = torch.tensor([[2, 9, 0],  # 样本 1 的目标序列
                        [0, 7, 3]]) # 样本 2 的目标序列
# 假设调用: model(idx, targets=targets)
```

**损失计算步骤 (在 `if targets is not None:` 内):**

1.  `B, T, C = logits.shape` -> `B=2`, `T=3`, `C=10`
2.  `logits_for_loss = logits.view(B * T, C)`: `logits` 变形为 `(6, 10)`。
    ```python
    # 变形后的 logits (形状: (6, 10)) - 来自上面步骤 8
    logits_for_loss = torch.tensor([
        [-0.1, 1.2, ..., 0.5], [-0.4, 0.8, ..., -0.1], [ 1.5, -0.9, ..., 0.2],
        [ 0.3, -1.1, ..., 0.9], [ 0.6, 0.2, ..., 1.4], [-0.8, 1.7, ..., -0.3]
    ])
    ```
3.  `targets_for_loss = targets.view(B * T)`: `targets` 变形为 `(6,)`。
    ```python
    # 变形后的 targets (形状: (6,))
    targets_for_loss = torch.tensor([2, 9, 0, 0, 7, 3]) # 注意：这里是提供的 targets 展平
    ```
4.  `loss = F.cross_entropy(logits_for_loss, targets_for_loss)`:
    计算交叉熵损失，输入 `(6, 10)` 的 logits 和 `(6,)` 的目标标签。
    *   **假设计算得到的 `loss`**:
        ```python
        loss = torch.tensor(2.153) # 示例损失值 (标量)
        ```

**正确逻辑下的返回值 (当 `targets` 被提供时):**

*   返回原始形状的 `logits` (形状 `(2, 3, 10)`)。
*   返回计算得到的标量 `loss` (例如 `tensor(2.153)`)。

In [7]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embedding_table = nn.Embedding(config.vocab_size, config.n_embd)
        self.position_embedding_table = nn.Embedding(config.block_size, config.n_embd)
        self.blocks = nn.Sequential(
            *[Block(config) for _ in range(config.n_layer)]
        )
        self.ln_final = nn.LayerNorm(config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        
        # linear (4 -> 8)； weight shape 是记上是 8 * 4，
        # 所以 embedding weight 和 lm_head weight 是共享的
        # 这里学习一下 tie weight。
        # 这是为了减少参数，加快训练；（现在 25的 SLM 很多都这样做了，注意⚠️）
        # self.token_embedding_table.weight = self.lm_head.weight

        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # 这里使用的是正态分布初始化
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        # idx 是输入的 token ids
        batch, seq_len = idx.size()
        token_emb = self.token_embedding_table(idx)

        # seq 长度是这次输入的最大长度
        pos_emb = self.position_embedding_table(
            # 要确保 位置编码和输入的 idx 在同一个设备上
            torch.arange(seq_len, device=idx.device)
        )
        # 有一个经典题目：为什么 embedding 和 position 可以相加？
        x = token_emb + pos_emb   # shape is (batch, seq_len, n_embd)
        x = self.blocks(x)
        x = self.ln_final(x)
        logits = self.lm_head(x)   # shape is (batch, seq_len, vocab_size)
        
        if targets is None:
            loss = None
        else:
            batch, seq_len, vocab_size = logits.size()
            logits = logits.view(batch * seq_len, vocab_size)
            targets = targets.view(batch * seq_len)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # 如果序列太长，只取最后 block_size 个token
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            # 获取预测
            logits, _ = self(idx_cond)
            # 只关注最后一个时间步的预测
            logits = logits[:, -1, :]  # becomes (B, vocab_size)
            # 应用softmax获取概率
            probs = F.softmax(logits, dim=-1)
            # 采样下一个token
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # 附加到序列上
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [8]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):
    def __init__(self, path, block_size = 512):
        import tiktoken
        self.enc = tiktoken.get_encoding("gpt2")
        self.block_size = block_size
        self.encoded_data = []
        self.eos_token = self.enc.encode(
            "<|endoftext|>",
            allowed_special={"<|endoftext|>"}
        )[0]
        self.max_lines = 1000
        import json
        raw_data = []
        with open(path, "r") as f:
            for i, line in enumerate(f):
                if i >= self.max_lines:
                    break
                try:
                    text = json.loads(line.strip())["text"]
                    raw_data.append(text)
                except Exception as e:
                    continue
        full_encoded = []
        for text in raw_data:
            encoded_text = self.enc.encode(text)
            full_encoded.extend(encoded_text + [self.eos_token])
        for i in range(0, len(full_encoded), self.block_size):
            chunk = full_encoded[i:i+self.block_size + 1]
            if len(chunk) < self.block_size + 1:
                chunk = chunk + [self.eos_token] * (self.block_size + 1 - len(chunk))
            self.encoded_data.append(chunk)

    def __len__(self):
        return len(self.encoded_data)
    def __getitem__(self, idx):
        chunk = self.encoded_data[idx]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y
    def encode(self, data):
        return self.enc.encode(data)
    def decode(self, data):
        return self.enc.decode(data)


In [9]:

# train data
train_dataset = Dataset('/home/m/data/mobvoi_seq_monkey_general_open_corpus.jsonl')

# split traindataset to train and val
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [0.9, 0.1])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [10]:
import torch_directml
model = GPT(GPTConfig())
device = torch_directml.device()
model.to(device)



total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params / 1e6}M")




optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000)

Total parameters: 162.643968M


In [ ]:
import time
import os 
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True) # 创建目录，如果不存在的话
def train(model, optimizer, scheduler, train_loader, val_loader, device):
    model.train()
    total_loss = 0.0
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, targets=y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')
    return total_loss
def eval(model, val_loader, device):
    # 验证
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits, loss = model(x, targets=y)
            val_loss += loss.item()
    return val_loss


for epoch in range(2):
    train_loss = train(model, optimizer, scheduler, train_loader, val_loader, device)
    val_loss = eval(model, val_loader, device)
    print(f'Epoch: {epoch}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

    # 保存模型
    avg_val_loss = val_loss / len(val_loader)
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'val_loss': avg_val_loss,
    }
    # 保存每个epoch的模型
    torch.save(checkpoint, f'checkpoints/model_epoch_{epoch}.pt')
    

Epoch: 0, Batch: 0, Loss: 10.9911
Epoch: 0, Batch: 100, Loss: 3.8353
Epoch: 0, Batch: 200, Loss: 3.5531
Epoch: 0, Batch: 300, Loss: 3.5043
Epoch: 0, Batch: 400, Loss: 0.6928


In [10]:
import torch
import torch.nn as nn
# ... (保持您其他的类定义，如 GPTConfig, SingleHeadAttention, MultiHeadAttention, FeedForward, Block, GPT) ...
import torch_directml
if __name__ == "__main__":
    # 1. 检查 DirectML 是否可用并设置设备

    config = GPTConfig()
    device = torch_directml.device()
    print(device)
    # 错误的方式 (这会导致错误)
    # device = torch.device("dml")
    # 假设 x 是一个张量
    x = torch.tensor([1., 2.])

    # 将 x 移动到 DirectML 设备
    x_on_dml = x.to(device)

    # 或者直接在设备上创建
    x_on_dml_direct = torch.tensor([1., 2.], device=device)
    # 假设 model 是您的 nn.Module 实例 (例如 gpt 模型)
    model = GPT(config) # 示例

    # 将模型的所有参数和缓冲区移动到 DirectML 设备
    model.to(device)

    # 创建输入张量 idx (二维) 并确保它在正确的设备上
    batch_size = 1
    seq_len = 10
    # 使用 torch.randint 创建随机索引，直接在目标设备上创建
    input_idx = torch.randint(0, config.vocab_size, (batch_size, seq_len), device=device)

    # 现在调用 forward
    model.forward(input_idx) # 传递二维的 input_idx
    print(model)

privateuseone:0


AttributeError: 'NoneType' object has no attribute 'view'